In [1]:
import tensorflow as tf

C:\Users\Miko Planas\.conda\envs\gans-action\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Miko Planas\.conda\envs\gans-action\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Miko Planas\.conda\envs\gans-action\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Miko Planas\.conda\envs\gans-action\l

In [2]:
print(tf.__version__)

1.12.0


In [5]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
import numpy as np

In [6]:
# hyperparams

batch_size = 100
original_dim = 28*28
latent_dim = 2
intermediate_dim = 256
nb_epoch = 5
epsilon_std = 1.0

In [9]:

# creaeting the sampler helper function
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0)
    return z_mean + K.exp(z_log_var / 2)*epsilon

# creating the encoder of the vae using functional api
# note that output_shape isnt necessary with the Tensorflow backend
x = Input(shape=(original_dim,), name="input")
h = Dense(intermediate_dim, activation='relu', name='encoding')(x)
z_mean = Dense(latent_dim, name="mean")(h)
z_log_var = Dense(latent_dim, name='log-variance')(h)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(x, [z_mean, z_log_var, z], name='encoder')


In [16]:
# decoder
input_decoder = Input(shape=(latent_dim,), name="decoder_input")
decoder_h = Dense(intermediate_dim, activation='relu', name='decoder_h')(input_decoder)
x_decoded = Dense(original_dim, activation='sigmoid', name="flat_decoded")(decoder_h)
decoder = Model (input_decoder, x_decoded, name="decoder")

# full vae
output_combined = decoder(encoder(x)[2])
vae = Model(x, output_combined)
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 784)               0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 201988    
_________________________________________________________________
decoder (Model)              (None, 784)               202256    
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [19]:
# define the loss function, overall loss of vae using kl divergence and binary cross entropy
# i think the format for this can be searched online
def vae_loss(x, x_decoded_mean,  original_dim=original_dim):
    xent_loss = original_dim* objectives.binary_crossentropy(x, x_decoded_mean)
    k1_loss = -0.5*K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=1)
    return xent_loss + k1_loss

vae.compile(optimizer="rmsprop", loss=vae_loss)

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [21]:
vae.fit(x_train, y_train, shuffle=True, nb_epoch=nb_epoch, batch_size=batch_size,
validation_data=(x_test, y_test), verbose=1)

C:\Users\Miko Planas\.conda\envs\gans-action\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 18s 306us/step - loss: -42935.7460 - val_loss: -43775.5054
Epoch 2/5
60000/60000 [==============================] - 14s 240us/step - loss: -43953.4445 - val_loss: -43953.7058
Epoch 3/5
60000/60000 [==============================] - 15s 253us/step - loss: -44140.2061 - val_loss: -44022.1359
Epoch 4/5
60000/60000 [==============================] - 15s 245us/step - loss: -44206.1579 - val_loss: -44071.5841
Epoch 5/5
60000/60000 [==============================] - 14s 239us/step - loss: -44217.3197 - val_loss: -44106.7252
